# Assembled Notebook — SK Agents (Cost Monitoring)
_Generated 2025-11-08T04:21:14.291568Z_

Notebook synthesized from your **Cost Monitoring Topology** Mermaid. Diagram is not embedded.

In [ ]:
# %% [SETUP]
!pip install -U semantic-kernel
!pip -q uninstall -y pydrive2
!pip install -q httpx


In [ ]:
# %% [SETUP-ENV]
import os, getpass
os.environ.setdefault('AZURE_OPENAI_ENDPOINT',    'https://4th-openai-resource.openai.azure.com')
os.environ.setdefault('AZURE_OPENAI_API_VERSION', '2024-10-21')
os.environ.setdefault('AZURE_OPENAI_DEPLOYMENT',  'gpt-35-turbo')
if not os.getenv('AZURE_OPENAI_API_KEY'):
    try:
        os.environ['AZURE_OPENAI_API_KEY'] = getpass.getpass('Enter AZURE_OPENAI_API_KEY (hidden): ').strip()
    except Exception:
        pass
print('Azure OpenAI env ready.')


In [ ]:
# %% [KERNEL]
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
kernel = Kernel()
service = AzureChatCompletion(
    service_id='azure',
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT'),
    endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
)
kernel.add_service(service)
print('Kernel ready (Azure OpenAI)')


In [ ]:
# %% [TOOLS]
import time, uuid, json
from typing import Dict, Any, Optional

POLICY_STORE: Dict[str, Dict[str, Any]] = {
    'tenant:acme': {
        'chat':    {'model': 'gpt-35-turbo', 'caps': {'aoai_tokens': 4000, 'rag_calls': 4, 'http_bytes': 200_000}, 'slo': {'p95_ms': 6000, 'usd_per_turn': 0.08}},
        'search':  {'model': 'gpt-35-turbo', 'caps': {'aoai_tokens': 2500, 'rag_calls': 6, 'http_bytes': 300_000}, 'slo': {'p95_ms': 8000, 'usd_per_turn': 0.05}},
        'default': {'model': 'gpt-35-turbo', 'caps': {'aoai_tokens': 3000, 'rag_calls': 3, 'http_bytes': 150_000}, 'slo': {'p95_ms': 7000, 'usd_per_turn': 0.06}},
    }
}

METER_STATE: Dict[str, Dict[str, Any]] = {}
PRICE = {'gpt-35-turbo': {'input': 0.001, 'output': 0.002}}

def tool_policy_lookup(tenant: str, intent: str) -> Dict[str, Any]:
    t = POLICY_STORE.get(tenant) or {}
    conf = t.get(intent) or t.get('default') or {}
    return {'tenant': tenant, 'intent': intent, 'policy': conf}

def _estimate_tokens(text: str) -> int:
    return max(1, int(len(str(text)) / 4))

def tool_meter_record(trace_id: str, kind: str, **metrics) -> Dict[str, Any]:
    st = METER_STATE.setdefault(trace_id, {'aoai_tokens': 0, 'rag_calls': 0, 'http_bytes': 0, 'events': []})
    st['events'].append({'t': time.time(), 'kind': kind, **metrics})
    st['aoai_tokens'] += int(metrics.get('tokens', 0))
    st['rag_calls'] += int(metrics.get('rag_calls', 0))
    st['http_bytes'] += int(metrics.get('http_bytes', 0))
    price = PRICE.get(metrics.get('model') or 'gpt-35-turbo', PRICE['gpt-35-turbo'])
    in_tok = int(metrics.get('input_tokens', 0)); out_tok = int(metrics.get('output_tokens', 0))
    usd = (in_tok * price['input'] + out_tok * price['output']) / 1000.0
    st['usd'] = st.get('usd', 0.0) + usd
    return {'ok': True, 'state': st}

def tool_cap_check(trace_id: str, caps: Dict[str, int]) -> Dict[str, Any]:
    st = METER_STATE.get(trace_id, {})
    viol = []
    for k, lim in (caps or {}).items():
        val = int(st.get(k, 0))
        if val > int(lim):
            viol.append({'key': k, 'value': val, 'limit': lim})
    return {'ok': len(viol) == 0, 'violations': viol, 'snapshot': st}

async def tool_aoai_infer(kernel, trace_id: str, prompt: str, model: Optional[str] = None) -> Dict[str, Any]:
    model = model or os.getenv('AZURE_OPENAI_DEPLOYMENT', 'gpt-35-turbo')
    in_tok = _estimate_tokens(prompt)
    try:
        result = await kernel.invoke_prompt(prompt, service_id='azure')
        out_text = str(result)
        out_tok = _estimate_tokens(out_text)
    except Exception as e:
        out_text = f"[AOAI stub due to error: {e}]"
        out_tok = _estimate_tokens(out_text)
    tool_meter_record(trace_id, 'aoai', model=model, input_tokens=in_tok, output_tokens=out_tok, tokens=in_tok+out_tok)
    return {'text': out_text, 'input_tokens': in_tok, 'output_tokens': out_tok, 'model': model}

def tool_rag_search(trace_id: str, query: str, k: int = 3) -> Dict[str, Any]:
    hits = [{'id': str(uuid.uuid4()), 'score': round(0.6 + 0.1*i, 3)} for i in range(k)]
    tool_meter_record(trace_id, 'rag', rag_calls=1)
    return {'hits': hits, 'k': k}

def tool_http_request(trace_id: str, method: str, url: str, body: Optional[str] = None) -> Dict[str, Any]:
    size = len(body or '') + len(url or '') + len(method or '')
    tool_meter_record(trace_id, 'http', http_bytes=size)
    return {'status': 200, 'bytes': size, 'url': url, 'method': method}

def tool_soap_call(trace_id: str, wsdl: str, action: str, payload: Optional[str] = None) -> Dict[str, Any]:
    size = len(payload or '') + len(wsdl or '') + len(action or '')
    tool_meter_record(trace_id, 'soap', http_bytes=size)
    return {'ok': True, 'action': action, 'bytes': size}

TOOLS = {
  'tool_policy_lookup': tool_policy_lookup,
  'tool_meter_record': tool_meter_record,
  'tool_cap_check': tool_cap_check,
  'tool_aoai_infer': tool_aoai_infer,
  'tool_rag_search': tool_rag_search,
  'tool_http_request': tool_http_request,
  'tool_soap_call': tool_soap_call,
}
print('Tools:', list(TOOLS.keys()))


In [ ]:
# %% [AGENTS]
from typing import List

class Agent_channel:
    def __init__(self, kernel):
        self.kernel = kernel
        self.name = 'Channel & Gateway'
        self.system_message = 'You are the Channel & Gateway agent.'
        self.skills = []
    async def run(self, user_text: str) -> str:
        try:
            res = await self.kernel.invoke_prompt(self.system_message + '\n\nUser: ' + str(user_text), service_id='azure')
            return str(res)
        except Exception as ex:
            return '[Channel & Gateway stub] ' + str(ex)

class Agent_planner:
    def __init__(self, kernel):
        self.kernel = kernel
        self.name = 'Planner (intent, plan, toolset)'
        self.system_message = 'Plan which tools to use and budgets to apply.'
        self.skills = ['tool_policy_lookup']
    async def run(self, user_text: str) -> str:
        try:
            res = await self.kernel.invoke_prompt(self.system_message + '\n\nUser: ' + str(user_text), service_id='azure')
            return str(res)
        except Exception as ex:
            return '[Planner stub] ' + str(ex)
    def call(self, tool_name: str, *args, **kwargs):
        return TOOLS[tool_name](*args, **kwargs)

class Agent_orchestrator:
    def __init__(self, kernel):
        self.kernel = kernel
        self.name = 'Orchestrator (SK / LangGraph)'
        self.system_message = 'Execute plan within budget caps.'
        self.skills = ['tool_aoai_infer','tool_rag_search','tool_http_request','tool_soap_call','tool_meter_record']
    async def run(self, user_text: str) -> str:
        try:
            res = await self.kernel.invoke_prompt(self.system_message + '\n\nUser: ' + str(user_text), service_id='azure')
            return str(res)
        except Exception as ex:
            return '[Orchestrator stub] ' + str(ex)
    def call(self, tool_name: str, *args, **kwargs):
        return TOOLS[tool_name](*args, **kwargs)

class Agent_meter:
    def __init__(self, kernel):
        self.kernel = kernel
        self.name = 'Token Meter'
        self.system_message = 'Track tokens, bytes, calls.'
        self.skills = ['tool_meter_record','tool_cap_check']
    async def run(self, user_text: str) -> str:
        try:
            res = await self.kernel.invoke_prompt(self.system_message + '\n\nUser: ' + str(user_text), service_id='azure')
            return str(res)
        except Exception as ex:
            return '[Meter stub] ' + str(ex)
    def call(self, tool_name: str, *args, **kwargs):
        return TOOLS[tool_name](*args, **kwargs)

class Agent_caps:
    def __init__(self, kernel):
        self.kernel = kernel
        self.name = 'Tool Budget Caps'
        self.system_message = 'Enforce per-tool caps.'
        self.skills = ['tool_cap_check']
    async def run(self, user_text: str) -> str:
        try:
            res = await self.kernel.invoke_prompt(self.system_message + '\n\nUser: ' + str(user_text), service_id='azure')
            return str(res)
        except Exception as ex:
            return '[Caps stub] ' + str(ex)
    def call(self, tool_name: str, *args, **kwargs):
        return TOOLS[tool_name](*args, **kwargs)

agent_channel = Agent_channel(kernel)
agent_planner = Agent_planner(kernel)
agent_orchestrator = Agent_orchestrator(kernel)
agent_meter = Agent_meter(kernel)
agent_caps = Agent_caps(kernel)
print('Agents:', ['agent_channel','agent_planner','agent_orchestrator','agent_meter','agent_caps'])


In [ ]:
# %% [WIRES]
WIRES = {
  'Channel & Gateway': [],
  'Planner (intent, plan, toolset)': ['tool_policy_lookup'],
  'Orchestrator (SK / LangGraph)': ['tool_aoai_infer','tool_rag_search','tool_http_request','tool_soap_call','tool_meter_record'],
  'Token Meter': ['tool_meter_record','tool_cap_check'],
  'Tool Budget Caps': ['tool_cap_check'],
}
print('Wiring entries:', len(WIRES))


In [ ]:
# %% [DEMO]
import asyncio, uuid

async def demo_cost_turn(tenant='tenant:acme', intent='chat', query='Summarize budget caps in one sentence.'):
    trace_id = str(uuid.uuid4())
    print('trace_id:', trace_id)

    pol = agent_planner.call('tool_policy_lookup', tenant=tenant, intent=intent)
    policy = pol.get('policy', {})
    caps = policy.get('caps', {})
    model = policy.get('model')
    print('policy:', policy)

    rag = agent_orchestrator.call('tool_rag_search', trace_id=trace_id, query='cost meter patterns', k=2)
    print('rag:', rag)

    prompt = 'You are a concise assistant. ' + query
    aoai = await TOOLS['tool_aoai_infer'](agent_orchestrator.kernel, trace_id=trace_id, prompt=prompt, model=model)
    print('aoai tokens:', aoai.get('input_tokens'), '->', aoai.get('output_tokens'))

    cap = agent_meter.call('tool_cap_check', trace_id=trace_id, caps=caps)
    if not cap.get('ok'):
        print('VIOLATIONS:', cap['violations'])
    else:
        print('Within caps \u2713')

    snap = METER_STATE.get(trace_id, {})
    print('meter snapshot:', {k: v for k, v in snap.items() if k != 'events'})
    return {'trace_id': trace_id, 'policy': policy, 'cap_ok': cap.get('ok'), 'meter': snap}

await demo_cost_turn()
